# Object Detection

Object detection is a phenomenon in computer vision that involves the detection of various objects in digital images or videos. Some of the objects detected include people, cars, chairs, stones, buildings, and animals.

This phenomenon seeks to answer two basic questions:

1. What is the object? This question seeks to identify the object in a specific image.
2. Where is it? This question seeks to establish the exact location of the object within the image.


#YOLO Introduction
YOLO is an algorithm that uses neural networks to provide real-time object etection. This algorithm is popular because of its speed and accuracy. It has
been used in various applications to detect traffic signals, people, parking meters, and animals. YOLO is an abbreviation for the term ‘You Only Look Once’. This is an algorithm that detects and recognizes various objects in a picture (in real-time). Object detection in YOLO is done as a regression problem and provides the class probabilities of the detected images.

First, the image is divided into various grids. Each grid has a dimension of S x S. The following image shows how an input image is divided into grids, in which there are many grid cells of equal dimension. Every grid cell will detect objects that appear within them.


![image](https://drive.google.com/uc?id=1VrRY7N7NOKKu-5NrjI-Gshy1CumVCoNX)

Then another technique will be used called Intersection over union (IOU), which is a phenomenon in object detection that describes how boxes overlap. YOLO uses IOU to provide an output box that surrounds the objects perfectly.

One issue that might happen is when the algorithm predicts several bounding boxes for one class. We could select only one box per class, that has the highest probability, but what if there are more objects of one class on the image (for example a few cats). Because of that,  a non-max suppression algorithm is used. First, we take the box with the maximum probability. After that, we compare the box with all other boxes of that particular class using IOU. If the IoU is higher than the predefined threshold (for example 0.5), then the box with a smaller probability is suppressed or excluded. It means that two boxes with high IoU values probably indicate the same object on the image, so we exclude the box with a lower probability. This process is repeated until all boxes are taken as object prediction or excluded.

![image](https://drive.google.com/uc?id=1kP80jh7iUI-OG4F1pGMDKEVoSmUxvuB7)

Let's test the pretrained yolo model on a sample image.

In [ ]:
!pip install ultralytics
from ultralytics import YOLO
!yolo predict model=yolov8n.pt source='https://ultralytics.com/images/bus.jpg'

# Fine Tuning Yolo on a new dataset

Now we want to use a new dataset to fine tune yolo on the dataset for using it for detecting some other kinds of object which may not exist in the origin dataset on which yolo is trained.

Here we want to fine tune yolo model on a Furniture dataset, which can be seen [here](https://universe.roboflow.com/mokhamed-nagy-u69zl/furniture-detection-qiufc/dataset/20). This dataset contains more than 7000 images of different furniture and their related annotations.

![image](https://drive.google.com/uc?id=1s3ZRTHVo8BRdgSjFAwgkefJqI5O86ek2)

First we download the dataset and prepare the data.

In [ ]:
!wget -O data.zip 'https://universe.roboflow.com/ds/yPJiNb0k0T?key=aZSRk47xb5'

In [ ]:
!unzip --qq '/content/data.zip'

The annotation of the data is like this:

![image](https://drive.google.com/uc?id=1s-qWMRNl0SU7jCtPkIYne2CLfjUawbHd)

Now we prepare the yolo model for the training procedure. (Make sure to modify the yaml file paths with the correct paths)

In [ ]:
!pip install ultralytics

In [ ]:
!yolo task=detect mode=train model=yolov8n.pt imgsz=640 data='/content/data.yaml' epochs=50 batch=86 name=yolov8n_furniture

Let's check the performance of the model on a sample image

In [ ]:
from ultralytics import YOLO
from PIL import Image

In [ ]:
names = ['Bed', 'Cabinet', 'Carpet', 'Ceramic floor', 'Chair', 'Closet', 'Cupboard', 'Curtains', 'Dining Table',
            'Door', 'Frame', 'Futec frame', 'Futech tiles', 'Gypsum Board', 'Lamp', 'Nightstand', 'Shelf', 'Sideboard',
            'Sofa', 'TV stand', 'Table', 'Transparent Closet', 'Wall Panel', 'Window', 'Wooden floor']

In [ ]:
weights_path = "/content/runs/detect/yolov8n_furniture/weights/best.pt"
model = YOLO(weights_path)

In [ ]:
img_path = '/content/test/images/-1-_jpg.rf.8fa48d61117eb985d8be50da70ee0911.jpg'
img = Image.open(img_path)
outputs = model.predict(img, conf=.1)[0]

In [ ]:
xyxys= []
confidences = []
classes = []
for result in outputs:
  boxes = result.boxes.cpu().numpy()
  xyxys.append(boxes.xyxy)
  confidences.append(boxes.conf)
  classes.append(boxes.cls)

In [ ]:
import cv2
from google.colab.patches import cv2_imshow
img = cv2.imread(img_path, cv2.IMREAD_UNCHANGED)
# print(img.shape)
for xy_ in xyxys:
  for idx , xy in enumerate(xy_):
    cv2.putText(img, text=str(confidences[0][idx]), org=(int(xy[0]) + 10,int(xy[1])+20), fontFace= cv2.FONT_HERSHEY_SIMPLEX, fontScale=.5, color=(0,255,0), thickness=1)
    cv2.putText(img, text=names[int(classes[0][idx])], org=(int(xy[0]) + 10,int(xy[1])+35), fontFace= cv2.FONT_HERSHEY_SIMPLEX, fontScale=.5, color=(0,255,0), thickness=1)
    cv2.rectangle(img, (int(xy[0]), int(xy[1])), (int(xy[2]), int(xy[3])), (0, 255, 0, 255), 2)
cv2_imshow(img)